<a href="https://colab.research.google.com/github/linhlinhle997/e2e-qa-distilbert/blob/develop/retrieval_extractive_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq datasets evaluate

In [ ]:
!sudo apt-get install -y libopenblas-dev
!pip install -q condacolab
import condacolab
condacolab.install()
!mamba install -c pytorch faiss-gpu -y

In [ ]:
import numpy as np
import collections
import torch
import faiss
import evaluate
from datasets import load_dataset
from tqdm.auto import tqdm

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    pipeline
)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Load Dataset

In [ ]:
raw_ds = load_dataset("squad_v2", split="train+validation").shard(num_shards=40, index=0)
raw_ds

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 3555
})

In [ ]:
test_datasets = load_dataset("squad_v2", split="validation")
test_datasets

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [ ]:
print(raw_ds["question"][0])
print(raw_ds["context"][0])
print(raw_ds["answers"][0])

When did Beyonce start becoming popular?
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
{'text': ['in the late 1990s'], 'answer_start': [269]}


Filter out non-answerable samples

In [ ]:
raw_ds = raw_ds.filter(lambda x: len(x["answers"]["text"]) > 0)
raw_ds

Filter:   0%|          | 0/3555 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 3207
})

Delete unnecessary columns

In [ ]:
columns = raw_ds.column_names

columns_to_keep = ["id", "context", "question", "answers"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
columns_to_remove

{'title'}

In [ ]:
raw_ds = raw_ds.remove_columns(columns_to_remove)
raw_ds

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 3207
})

## Init pretrained model

In [ ]:
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
pipeline_name = "question-answering"
qa_model_name = "linhlinhle997/distilbert-finetuned-squadv2"

qa_pipeline  = pipeline(pipeline_name, model=qa_model_name)

config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


### Create vector embedding

In [ ]:
def get_embeddings(text_list):
    with torch.no_grad():
        encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        model_output = model(**encoded_input)

    return model_output.last_hidden_state[:, 0].cpu().numpy() # Only get token <cls>

## Embedding Question

### Create vector embedding for question

In [ ]:
# Test funciton
embedding = get_embeddings(raw_ds["question"][0])
embedding.shape

(1, 768)

In [ ]:
batch_size = 32
embedding_column = "question_embedding"

embedding_ds = raw_ds.map(
    lambda batch: {embedding_column: get_embeddings(batch["question"])},
    batched=True,
    batch_size=batch_size
)

embedding_ds.add_faiss_index(column=embedding_column)

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'context', 'question', 'answers', 'question_embedding'],
    num_rows: 3207
})

### QA

#### Find similar samples based on the given question.

In [ ]:
input_question = "When did Beyonce start becoming popular?"

input_question_embedding = get_embeddings([input_question])
input_question_embedding.shape

(1, 768)

In [ ]:
top_k = 3

scores, samples = embedding_ds.get_nearest_examples(
    embedding_column,
    input_question_embedding,
    k=top_k
)

In [ ]:
for idx, score in enumerate(scores):
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Question: {samples["question"][idx]}')
    print(f'Context: {samples["context"][idx]}')
    print()

Top 1	Score: 5.5387507080784104e-11
Question: When did Beyonce start becoming popular?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

Top 2	Score: 2.6135330200195312
Question: When did Beyoncé rise to fame?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record p

#### Use the QA model to extract the answer from the retrieved Context.

In [ ]:
print(f'Input question: {input_question}')
for idx, score in enumerate(scores):
    context = samples["context"][idx]
    answer = qa_pipeline(question=input_question, context=context)

    print(f'Top {idx + 1} | Score: {score:.4f}')
    print(f'Context: {context}')
    print(f'Answer: {answer}')
    print()

Input question: When did Beyonce start becoming popular?
Top 1 | Score: 0.0000
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Answer: {'score': 0.5104944705963135, 'start': 276, 'end': 286, 'answer': 'late 1990s'}

Top 2 | Score: 2.6135
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American

### Test

In [ ]:
top_k = 3

for idx, input_question in enumerate(test_question):
    input_question_embedding = get_embeddings([input_question])

    scores, samples = embedding_ds.get_nearest_examples(
        embedding_column, input_question_embedding, k=top_k
    )

    print(f"Question: {idx+1}: {input_question}\n")
    for jdx, scores in enumerate(scores):
        context = samples["context"][jdx]
        answer = qa_pipeline(question=input_question, context=context)

        print(f"Top {jdx+1} | Score: {score:.4f}")
        print(f"Context: {context}")
        print(f"Answer: {answer}")
        print()
    print("="*100)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: 1: How many awards was Beyonce nominated for at the 52nd Grammy Awards?

Top 1 | Score: 4.8595
Context: At the 52nd Annual Grammy Awards, Beyoncé received ten nominations, including Album of the Year for I Am... Sasha Fierce, Record of the Year for "Halo", and Song of the Year for "Single Ladies (Put a Ring on It)", among others. She tied with Lauryn Hill for most Grammy nominations in a single year by a female artist. In 2010, Beyoncé was featured on Lady Gaga's single "Telephone" and its music video. The song topped the US Pop Songs chart, becoming the sixth number-one for both Beyoncé and Gaga, tying them with Mariah Carey for most number-ones since the Nielsen Top 40 airplay chart launched in 1992. "Telephone" received a Grammy Award nomination for Best Pop Collaboration with Vocals.
Answer: {'score': 0.8438270092010498, 'start': 51, 'end': 54, 'answer': 'ten'}

Top 2 | Score: 4.8595
Context: At the 57th Annual Grammy Awards in February 2015, Beyoncé was nominated for six

## Embedding Context

In [ ]:
batch_size = 32
embedding_column = "context_embedding"

embedding_ds = raw_ds.map(
    lambda batch: {embedding_column: get_embeddings(batch["context"])},
    batched=True,
    batch_size=batch_size
)

embedding_ds.add_faiss_index(column=embedding_column)

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'context', 'question', 'answers', 'context_embedding'],
    num_rows: 3207
})

### QA

#### Find similar samples based on the given question.

In [ ]:
input_question = "When did Beyonce start becoming popular?"
top_k = 3

input_question_embedding = get_embeddings([input_question])

scores, samples = embedding_ds.get_nearest_examples(
    embedding_column,
    input_question_embedding,
    k=top_k
)

for idx, score in enumerate(scores):
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Question: {samples["question"][idx]}')
    print(f'Context: {samples["context"][idx]}')
    print()

Top 1	Score: 31.23427391052246
Question: Beyonce than appeared again on the Time 100 list in what year?
Context: In The New Yorker music critic Jody Rosen described Beyoncé as "the most important and compelling popular musician of the twenty-first century..... the result, the logical end point, of a century-plus of pop." When The Guardian named her Artist of the Decade, Llewyn-Smith wrote, "Why Beyoncé? [...] Because she made not one but two of the decade's greatest singles, with Crazy in Love and Single Ladies (Put a Ring on It), not to mention her hits with Destiny's Child; and this was the decade when singles – particularly R&B singles – regained their status as pop's favourite medium. [...] [She] and not any superannuated rock star was arguably the greatest live performer of the past 10 years." In 2013, Beyoncé made the Time 100 list, Baz Luhrmann writing "no one has that voice, no one moves the way she moves, no one can hold an audience the way she does... When Beyoncé does an alb

#### Use the QA model to extract the answer from the retrieved Context.

In [ ]:
print(f'Input question: {input_question}')
for idx, score in enumerate(scores):
    context = samples["context"][idx]
    answer = qa_pipeline(question=input_question, context=context)

    print(f'Top {idx + 1} | Score: {score:.4f}')
    print(f'Context: {context}')
    print(f'Answer: {answer}')
    print()

Input question: When did Beyonce start becoming popular?
Top 1 | Score: 31.2343
Context: In The New Yorker music critic Jody Rosen described Beyoncé as "the most important and compelling popular musician of the twenty-first century..... the result, the logical end point, of a century-plus of pop." When The Guardian named her Artist of the Decade, Llewyn-Smith wrote, "Why Beyoncé? [...] Because she made not one but two of the decade's greatest singles, with Crazy in Love and Single Ladies (Put a Ring on It), not to mention her hits with Destiny's Child; and this was the decade when singles – particularly R&B singles – regained their status as pop's favourite medium. [...] [She] and not any superannuated rock star was arguably the greatest live performer of the past 10 years." In 2013, Beyoncé made the Time 100 list, Baz Luhrmann writing "no one has that voice, no one moves the way she moves, no one can hold an audience the way she does... When Beyoncé does an album, when Beyoncé sings a

### Test

In [ ]:
test_question = embedding_ds["question"][200:210]
test_question

['How many awards was Beyonce nominated for at the 52nd Grammy Awards?',
 'Beyonce tied with which artist for most nominations by a female artist?',
 'In 2010, Beyonce worked with which other famous singer?',
 'How many number one singles did Beyonce now have after the song "Telephone"?',
 'Beyonce tied who for most number one singles by a female?',
 'Beyonce received how many nominations at the 52nd Annual Grammy Awards?',
 'What song was the sixth first place song for Beyonce?',
 'Who else appeared with Beyonce in Telephone?',
 'Who did they tie with for six top songs?',
 'Who did Beyonce tie with for the most nominations in a year?']

In [ ]:
test_datasets = load_dataset("squad_v2", split="validation")
test_question = embedding_ds["question"][200:210]

top_k = 3

for idx, input_question in enumerate(test_question):
    input_question_embedding = get_embeddings([input_question])

    scores, samples = embedding_ds.get_nearest_examples(
        embedding_column, input_question_embedding, k=top_k
    )

    print(f"Question: {idx+1}: {input_question}\n")
    for jdx, scores in enumerate(scores):
        context = samples["context"][jdx]
        answer = qa_pipeline(question=input_question, context=context)

        print(f"Top {jdx+1} | Score: {score:.4f}")
        print(f"Context: {context}")
        print(f"Answer: {answer}")
        print()
    print("="*100)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Question: 1: How many awards was Beyonce nominated for at the 52nd Grammy Awards?

Top 1 | Score: 31.2343
Context: In The New Yorker music critic Jody Rosen described Beyoncé as "the most important and compelling popular musician of the twenty-first century..... the result, the logical end point, of a century-plus of pop." When The Guardian named her Artist of the Decade, Llewyn-Smith wrote, "Why Beyoncé? [...] Because she made not one but two of the decade's greatest singles, with Crazy in Love and Single Ladies (Put a Ring on It), not to mention her hits with Destiny's Child; and this was the decade when singles – particularly R&B singles – regained their status as pop's favourite medium. [...] [She] and not any superannuated rock star was arguably the greatest live performer of the past 10 years." In 2013, Beyoncé made the Time 100